In [0]:
import logging
from typing import Tuple
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col
from mdd.utils import DecoratorUtil, DeltaTableUtil, FunctionUtil
from mdd.metadata import Metadata
from mdd.datareader import DeltaTableReader
from mdd.datawriter import DeltaTableWriter


@DecoratorUtil.add_logger()
class TransformDataFlow:
    logger: logging.Logger

    def __init__(self, spark: SparkSession, metadata_yml: str):
        self.spark = spark
        self.metadata = Metadata(metadata_yml, False)
        self.debug = self.metadata.get("debug")
        self.active = self.metadata.get("active")
        self.source_name = self.metadata.get("reader", "source_name")
        self.sink_name = self.metadata.get("writer", "sink_name")

        dataflow_type = self.metadata.get("dataflow_type")
        if dataflow_type != "transform":
            message = (
                f"Invalid dataflow type: {dataflow_type}, it should be 'transform'"
            )
            self.logger.error(message)
            raise Exception(message)

        DeltaTableUtil.ensure_system_columns(self.spark, self.sink_name)

    @DecoratorUtil.log_function()
    def read_stream(self) -> Tuple[DataFrame, str]:
        """
        Reads data from the specified Delta table using the provided configuration.

        :return: Spark DataFrame containing the read data
        """
        mode = self.metadata.get("sync_options", "mode")
        backfill_days = self.metadata.get("sync_options", "backfill_days")

        # build the config for data reader
        config = {}
        config["source_name"] = self.source_name
        config["mode"] = mode
        config["backfill_days"] = backfill_days
        if mode == "full":
            # get the max _source_timestamp from the target table
            max_source_timestamp = DeltaTableUtil.get_max_column_value(
                self.spark, self.sink_name, "_source_timestamp"
            )
            config["full_max_processed_timestamp"] = max_source_timestamp
        elif mode == "incremental":
            max_source_commit_version = DeltaTableUtil.get_max_column_value(
                self.spark,
                "mdd.table_control",
                "source_commit_version",
                f"table_name = '{self.sink_name}' and source_name = '{self.source_name}'",
            )
            config["incremental_max_processed_version"] = max_source_commit_version
        else:
            message = f"Invalid mode: {mode}"
            self.logger.error(message)
            raise ValueError(message)

        if self.debug:
            self.logger.info(f"Config: {config}")

        reader = DeltaTableReader(spark=self.spark, config=config, debug=self.debug)
        df, sync_mode = reader.read_stream()

        return df, sync_mode

    @DecoratorUtil.log_function()
    def transform(self, df: DataFrame) -> DataFrame:
        return df

    @DecoratorUtil.log_function()
    def write_stream(self, df: DataFrame):
        DeltaTableUtil.ensure_system_columns(self.spark, self.sink_name)
        
        config_writer = self.metadata.get("writer")
        writer = DeltaTableWriter(self.spark, config_writer, self.debug)
        query = writer.write_stream(df)
        query.awaitTermination()

        # run post script
        writer.execute_postscript()

    @DecoratorUtil.log_function()
    def run(self):
        self.logger.info("Transform start ...")

        self.logger.info("Read start...")
        df, sync_mode = self.read_stream()
        self.logger.info("Read end...")

        self.logger.info("Deduplicate start...")
        df = self.transform(df)
        self.logger.info("Deduplicate end...")

        self.logger.info("Write start...")
        self.write_stream(df, sync_mode)
        self.logger.info("Write end...")

In [0]:
%load_ext autoreload
%autoreload 2
import logging
import datetime
from pyspark.sql.functions import lit
from mdd.logger import *
#from mdd.transformer import *

log_folder = "mdd_test"
log_file_name = "test_transformer"
log_timestamp = datetime.datetime.now()
debug = False
Logger.init(log_folder, log_file_name, log_timestamp, debug)

try:
    metadata_yml = "transform/gold_fact_combinedcards.yml"
    dataflow = TransformDataFlow(spark, metadata_yml)
    df_read, sync_mode = dataflow.read_stream()
    print(f"sync_mode: {sync_mode}")
except Exception as e:
    print(e)
finally:
    logging.shutdown()



In [0]:
display(df_read)


In [0]:
df_read.isStreaming

In [0]:
df_deduplicated = dataflow.deduplicate(df_read)
display(df_deduplicated)

In [0]:
%sql
select count(*) from lakehouse.bronze.paytronix_mid352_combinedcards;

In [0]:
drop_columns = ["_corrupt_record", "_rescued_data", "_mode", "_change_type", "_commit_version", "_commit_timestamp"]

from mdd.helper.deltatable import DeltaTableUtils as dtu
df_transformed = DeltaTableUtil.safe_drop_columns(df_read, drop_columns)
display(df_transformed)

In [0]:
df_transformed.isStreaming

In [0]:
dataflow.write_stream(df_transformed)

In [0]:
%sql
update lakehouse.bronze.paytronix_mid352_combinedcards
set card_template = 'visa4', `_record_timestamp`=current_timestamp()
where primary_card_number = '6000201936678524';

select * from lakehouse.bronze.paytronix_mid352_combinedcards
where primary_card_number = '6000201936678524';   

--describe history lakehouse.bronze.paytronix_mid352_combinedcards

In [0]:
%sql
select * from lakehouse.bronze.paytronix_mid352_combinedcards
where primary_card_number = '6000201936678524';   

In [0]:
df = spark.readStream.table("bronze.paytronix_mid352_combinedcards")
#display(df.filter("primary_card_number = '6000201936678524'"))
df = df.filter("_record_timestamp >= '2025-05-25'").filter("primary_card_number = '6000201936678524'")
#display(df)

In [0]:
import shutil

checkpoint_path = "/dbfs//tmp/checkpoint5"

shutil.rmtree(checkpoint_path, ignore_errors=True)

In [0]:
df.writeStream.format("delta").trigger(availableNow=True).option("checkpointLocation", "/tmp/checkpoint5").table("silver.paytronix_mid352_combinedcards")

In [0]:
%sql
select * from silver.paytronix_mid352_combinedcards
where primary_card_number = '6000201936678524'